# Recommending types of businesses to open in different Neighborhoods in Toronto

In [52]:
# importing packages
import numpy as np 
import pandas as pd
from sklearn.cluster import KMeans


In [53]:
# importing pakage for web scraping
!pip install beautifulSoup4 
!pip install lxml # import package for reading xml 
!pip install requests #Requests allows you to send organic, grass-fed HTTP/1.1 requests,

You should consider upgrading via the 'python -m pip install --upgrade pip' command.
ERROR: Invalid requirement: '#'
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
ERROR: Invalid requirement: '#Requests'
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


parsers to parse html

In [3]:
#  import the installed packages
from bs4 import BeautifulSoup
import requests 

In [54]:
# read the data
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [55]:
soup=BeautifulSoup(website_url,'lxml')

In [56]:
# to see the data clearly with indentation and tags
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XiXg7gpAICMAAKBeVo0AAADE","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":935851093,"wgRevisionId":935851093,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

In [57]:
# this is found in the above output as the place where the table is created in the webpage.. 
# to access the table use the class and name of the class
table=soup.find('table',{'class':'wikitable sortable'})

In [58]:
print(table) # view the table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [59]:
links=table.find_all('a')#Now to extract all the links within <a>, we will use find_all().
post=table.find_all('tr') #similarly for tr

In [60]:
# this code creates the df with columns and rows fetched from the table..    
    
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
#             for col in df:
#                 try:
#                     df[col] = df[col].astype(float)
#                 except ValueError:
#                     pass

In [61]:
# removing the newline added in the column name
df['Neighbourhood']=df['Neighbourhood\n']

In [62]:
# remove the \n from data for the column as well
for i in df['Neighbourhood']:
    df['Neighbourhood']=df.Neighbourhood.str.replace('\n','')


In [63]:
df=df[df['Borough']!='Not assigned']  #remove all not assigned cells from column

In [64]:
# find rows where neighbourhood is not assigned
idx_to_change=df.loc[df['Neighbourhood']=='Not assigned']
# print(idx_to_change['Neighbourhood'])

# rename the not assigned go borough column value
for i in idx_to_change:
    idx_to_change['Neighbourhood']=idx_to_change['Borough']

# compare neighbourhood\n(old column) with neighbourhood(new column with Borough cell value)
print(idx_to_change)


    

  Postcode       Borough Neighbourhood\n Neighbourhood
9      M9A  Queen's Park  Not assigned\n  Queen's Park


C:\Users\DR36495\AnacondaWithSpyder\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [65]:
df.drop(['Neighbourhood\n'],axis=1,inplace=True)# this column was an extra column so removing it


In [66]:
# df1=df.groupby(['Postcode','Borough']).apply(lambda x:','.join(x.Neighbourhood)) This does not work but good for 
df.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ','.join(x))

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Downtown Toronto,Queen's Park
M9A,Queen's Park,Not assigned
M1B,Scarborough,"Rouge,Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens,Parkview Hill"


In [67]:
df.shape

(210, 3)

In [16]:
!pip install geocoder

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [68]:
import geocoder
!pip install geopy
from geopy.geocoders import Nominatim

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [69]:
from pandas.io.json import json_normalize 

In [70]:
# check lon lan for one Scarborough
address = 'Scarborough	'

geolocator = Nominatim(user_agent="Canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 54.2820009, -0.4011868.


In [76]:
df['latitude']=None
df['longitude']=None
for i,rows in df.iterrows():
    lat_lan=None
    while (lat_lan==None):
        g=geocoder.arcgis(rows['Borough'].format(df['Postcode']))
        lat_lan=g.x+g.y
        df.loc[i,'latitude']=g.y
        df.loc[i,'longitude']=g.x


In [78]:
import folium
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[54.2820009, -0.4011868], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [79]:
df.reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood,latitude,longitude
2,M3A,North York,Parkwoods,43.7683,-79.4126
3,M4A,North York,Victoria Village,43.7683,-79.4126
4,M5A,Downtown Toronto,Harbourfront,43.6501,-79.3829
5,M6A,North York,Lawrence Heights,43.7683,-79.4126
6,M6A,North York,Lawrence Manor,43.7683,-79.4126


In [80]:

# df['latitude']=None
# df['longitude']=None
# for i,j in df.iterrows():
#     address=j['Borough']
#     geolocator = Nominatim(user_agent="Canada_explorer")
    
#     location = geolocator.geocode(address)
#     df.loc[i,'latitude'] = location.latitude
#     df.loc[i,'longitude'] = location.longitude
    #print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

In [81]:
!pip install folium

df.head()

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


,Postcode,Borough,Neighbourhood,latitude,longitude
2,M3A,North York,Parkwoods,43.7683,-79.4126
3,M4A,North York,Victoria Village,43.7683,-79.4126
4,M5A,Downtown Toronto,Harbourfront,43.6501,-79.3829
5,M6A,North York,Lawrence Heights,43.7683,-79.4126
6,M6A,North York,Lawrence Manor,43.7683,-79.4126


In [82]:
import folium
df=df[df['Borough'].str.contains('Toronto')]


In [83]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 4 boroughs and 74 neighborhoods.


In [84]:
# create map of Toronto Map using latitude and longitude values

map_ny=folium.Map(location=[-80.73502252,  43.65011 ],zoom_start=10)

# add markers to the map
for lat, lon, borough, neighbor in zip(df2['latitude'],df2['longitude'],df2['Borough'],df2['Neighbourhood']):
#     print(lat, lon,borough)
    label='{},{}'.format(neighbor,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny
    
    

In [85]:
CLIENT_ID='IUQMV40FTE1IMIAEOBV1SDMIFYTLV242NGKDWYVYKTCMGJVB'

CLIENT_SECRET='0EH4DMV3I5SY3M5GA51KZOCI4DBGGCQWIWY4IIUXMZY3PMV2'
VERSION='20190916'
LIMIT =100



In [91]:
df['latitude'] 

4      43.6501
7      43.6501
16     43.6501
17     43.6501
33     43.6501
        ...   
249    43.6501
250    43.6501
251    43.6501
261    43.6501
264     43.659
Name: latitude, Length: 74, dtype: object

In [92]:
neighborhood_latitude = df.loc[264,'latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[264,'longitude'] # neighborhood longitude value

neighborhood_name = df.loc[264, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                            
                                                               neighborhood_longitude))
LIMIT = 100 
radius = 1000

Latitude and longitude values of Business Reply Mail Processing Centre 969 Eastern are 43.65902991949926, -79.34900993060764.


In [93]:


url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()
# results = requests.get('https://mytestproject.com').json()

normal_results=json_normalize(results)

normal_results.columns


Index(['meta.code', 'meta.requestId', 'response.venues', 'response.confident'], dtype='object')

In [94]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [95]:

from pandas.io.json import json_normalize

In [96]:
venues = results['response']['venues'][0]


nearby_venues=json_normalize(venues)#flatten JSON

print(nearby_venues.columns)
    
# # filter columns
filtered_columns = ['name','categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# # filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
print(nearby_venues)
# # clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

Index(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.address',
       'location.lat', 'location.lng', 'location.labeledLatLngs',
       'location.distance', 'location.postalCode', 'location.cc',
       'location.city', 'location.state', 'location.country',
       'location.formattedAddress', 'venuePage.id'],
      dtype='object')
             name                                         categories  \
0  est Restaurant  [{'id': '4bf58dd8d48988d14e941735', 'name': 'A...   

   location.lat  location.lng     venue.categories  
0     43.658911    -79.349035  American Restaurant  


,name,categories,lat,lng,categories
0,est Restaurant,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",43.658911,-79.349035,American Restaurant


In [97]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


# Part 2 Explore neighborhood for Etobicoke Borough

In [103]:
# # create df for etobicoke called as Etobicoke
Etobicoke = df[df['Borough'].isin(['Downtown Toronto','East Toronto','West Toronto','Central Toronto'])].reset_index(drop=True)
Etobicoke.head()

Etobicoke=df.reset_index(drop=True)
Etobicoke.head()
# df['Borough'].unique()

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M5A,Downtown Toronto,Harbourfront,43.6501,-79.3829
1,M7A,Downtown Toronto,Queen's Park,43.6501,-79.3829
2,M5B,Downtown Toronto,Ryerson,43.6501,-79.3829
3,M5B,Downtown Toronto,Garden District,43.6501,-79.3829
4,M5C,Downtown Toronto,St. James Town,43.6501,-79.3829


Create a function to to repeat the same process as above to all neighbourrhood for etobicoke

In [104]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
 # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
       

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now is the code to call the above function for each neighbourhood call the new datafame all_venues

In [105]:
# type your answer here

all_venues = getNearbyVenues(names=Etobicoke['Neighbourhood'],
                                   latitudes=Etobicoke['latitude'],
                                   longitudes=Etobicoke['longitude']
                                  )



Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city

In [106]:
# check the size of all_venues
# check head of all_vernues
print(all_venues.shape)
all_venues.head()

(4259, 7)


,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65011,-79.3829,The Keg Steakhouse + Bar,43.649937,-79.384196,Steakhouse
1,Harbourfront,43.65011,-79.3829,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
2,Harbourfront,43.65011,-79.3829,Cactus Club Cafe,43.649552,-79.381671,American Restaurant
3,Harbourfront,43.65011,-79.3829,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
4,Harbourfront,43.65011,-79.3829,Pilot Coffee Roasters,43.648835,-79.380936,Coffee Shop


# Analyse each neighbourhood

In [107]:
# check number of unique neighbourhoods
all_venues['Neighborhood'].unique()
all_venues.columns

Index(['Neighborhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [108]:
#  one hot encoding creating dummies 
Etobicoke_onehot=pd.get_dummies(all_venues[['Venue Category']], prefix="",prefix_sep="")
print(Etobicoke_onehot)

      American Restaurant  Art Gallery  Asian Restaurant  BBQ Joint  Bakery  \
0                       0            0                 0          0       0   
1                       0            0                 0          0       0   
2                       1            0                 0          0       0   
3                       0            0                 0          0       0   
4                       0            0                 0          0       0   
...                   ...          ...               ...        ...     ...   
4254                    0            0                 0          0       0   
4255                    0            0                 0          0       0   
4256                    0            0                 0          0       0   
4257                    0            0                 0          0       0   
4258                    0            0                 0          0       0   

      Bar  Beer Bar  Big Box Store  Bookstore  Braz

In [109]:
# adding neighbourhood column back to this new dataframe
Etobicoke_onehot['Neighbourhood']=all_venues['Neighborhood']
# move neighbourhood column to the first column

fixed_columns = [Etobicoke_onehot.columns[-1]] + list(Etobicoke_onehot.columns[:-1])
Etobicoke_onehot=Etobicoke_onehot[fixed_columns]
# check head
Etobicoke_onehot.head()


,Neighbourhood,American Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Big Box Store,Bookstore,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Harbourfront,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
# Examine th size of neew dataframe
Etobicoke_onehot.shape

(4259, 86)

In [111]:
# lets lets group rows by neighbourhood and take mean of the freq of occurance of each category
Etobicoke_grouped=Etobicoke_onehot.groupby('Neighbourhood').mean().reset_index()
Etobicoke_grouped.head()

,Neighbourhood,American Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Big Box Store,Bookstore,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Adelaide,0.030000,0.01,0.03,0.0,0.020000,0.030000,0.01,0.0,0.02,...,0.01,0.010000,0.01,0.0,0.0,0.01,0.000000,0.010000,0.01,0.000000
1,Bathurst Quay,0.030000,0.01,0.03,0.0,0.020000,0.030000,0.01,0.0,0.02,...,0.01,0.010000,0.01,0.0,0.0,0.01,0.000000,0.010000,0.01,0.000000
2,Berczy Park,0.030000,0.01,0.03,0.0,0.020000,0.030000,0.01,0.0,0.02,...,0.01,0.010000,0.01,0.0,0.0,0.01,0.000000,0.010000,0.01,0.000000
3,Brockton,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.00,0.0,0.00,...,0.00,0.000000,0.00,1.0,0.0,0.00,0.000000,0.000000,0.00,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.045455,0.00,0.00,0.0,0.045455,0.068182,0.00,0.0,0.00,...,0.00,0.022727,0.00,0.0,0.0,0.00,0.022727,0.022727,0.00,0.022727


In [112]:
# lets print each neighbourhood and the 5 most common venues

num_top_venues=5

for hood in Etobicoke_grouped['Neighbourhood']:
    print('*****'+hood+'*****')
    temp=Etobicoke_grouped[Etobicoke_grouped['Neighbourhood']==hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    

*****Adelaide*****
          venue  freq
0   Coffee Shop  0.07
1          Café  0.06
2         Hotel  0.04
3  Burger Joint  0.03
4    Steakhouse  0.03


*****Bathurst Quay*****
          venue  freq
0   Coffee Shop  0.07
1          Café  0.06
2         Hotel  0.04
3  Burger Joint  0.03
4    Steakhouse  0.03


*****Berczy Park*****
          venue  freq
0   Coffee Shop  0.07
1          Café  0.06
2         Hotel  0.04
3  Burger Joint  0.03
4    Steakhouse  0.03


*****Brockton*****
                 venue  freq
0     Toy / Game Store   1.0
1  American Restaurant   0.0
2         Neighborhood   0.0
3                  Pub   0.0
4                Plaza   0.0


*****Business Reply Mail Processing Centre 969 Eastern*****
                 venue  freq
0                  Bar  0.07
1          Coffee Shop  0.07
2  American Restaurant  0.05
3                 Café  0.05
4              Brewery  0.05


*****CN Tower*****
          venue  freq
0   Coffee Shop  0.07
1          Café  0.06
2         Hotel  

4              Brewery  0.05


*****Roncesvalles*****
                 venue  freq
0     Toy / Game Store   1.0
1  American Restaurant   0.0
2         Neighborhood   0.0
3                  Pub   0.0
4                Plaza   0.0


*****Rosedale*****
          venue  freq
0   Coffee Shop  0.07
1          Café  0.06
2         Hotel  0.04
3  Burger Joint  0.03
4    Steakhouse  0.03


*****Roselawn*****
                 venue  freq
0  American Restaurant  0.07
1         Burger Joint  0.07
2       Ice Cream Shop  0.07
3               Lounge  0.07
4        Grocery Store  0.07


*****Runnymede*****
                 venue  freq
0     Toy / Game Store   1.0
1  American Restaurant   0.0
2         Neighborhood   0.0
3                  Pub   0.0
4                Plaza   0.0


*****Ryerson*****
          venue  freq
0   Coffee Shop  0.07
1          Café  0.06
2         Hotel  0.04
3  Burger Joint  0.03
4    Steakhouse  0.03


*****South Hill*****
                 venue  freq
0  American Restaurant  

Putting the above in a pandas dataframe
Function to sort the venues in descending order.

In [113]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
# Now let's create the new dataframe and display the top 5 venues for each neighborhood.

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Etobicoke_grouped['Neighbourhood']

for ind in np.arange(Etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()



,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adelaide,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
1,Bathurst Quay,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
2,Berczy Park,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
3,Brockton,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega
4,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Bar,American Restaurant,Café,Pizza Place


# Clustering Neighbours

In [115]:
#  run k-means to cluster the neighbours
# set number of clusters
kclusters = 5

Etobicoke_grouped_clustering1 = Etobicoke_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Etobicoke_grouped_clustering1)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5]

C:\Users\DR36495\AnacondaWithSpyder\lib\site-packages\sklearn\cluster\k_means_.py:972: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([2, 2, 2, 1, 3])

In [116]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Etobicoke_merged = Etobicoke

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Etobicoke_merged = Etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Etobicoke_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.6501,-79.3829,2,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
1,M7A,Downtown Toronto,Queen's Park,43.6501,-79.3829,2,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
2,M5B,Downtown Toronto,Ryerson,43.6501,-79.3829,2,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
3,M5B,Downtown Toronto,Garden District,43.6501,-79.3829,2,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
4,M5C,Downtown Toronto,St. James Town,43.6501,-79.3829,2,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant


In [118]:
import matplotlib.cm as cm

import matplotlib.colors as colors
map_clusters= folium.Map(location=[latitude,longitude],zoom_start=11)

# set color scheme for the clusters
x=np.arange(kclusters)
ys=[i+x+(i+x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

#  add markers to the map


markers_color=[]
for lat, lon,poi,cluster in zip(Etobicoke_merged['latitude'],Etobicoke_merged['longitude'],Etobicoke_merged['Neighbourhood'],Etobicoke_merged['Cluster Labels']):
    label=folium.Popup(str(poi)+'Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.7).add_to(map_clusters
#     http://localhost:8888/notebooks/Capstone%20w3%20Part2%20.ipynb#
        
        
map_clusters                       

SyntaxError: invalid syntax (<ipython-input-118-aaa44758e992>, line 29)

# Examine the Clusters individually

In [119]:
Etobicoke_merged.head()

,Postcode,Borough,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.6501,-79.3829,2,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
1,M7A,Downtown Toronto,Queen's Park,43.6501,-79.3829,2,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
2,M5B,Downtown Toronto,Ryerson,43.6501,-79.3829,2,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
3,M5B,Downtown Toronto,Garden District,43.6501,-79.3829,2,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant
4,M5C,Downtown Toronto,St. James Town,43.6501,-79.3829,2,Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant


In [120]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 0, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
31,Central Toronto,0,American Restaurant,Big Box Store,Pizza Place,Cosmetics Shop,Grocery Store
32,Central Toronto,0,American Restaurant,Big Box Store,Pizza Place,Cosmetics Shop,Grocery Store
33,Central Toronto,0,American Restaurant,Big Box Store,Pizza Place,Cosmetics Shop,Grocery Store
34,Central Toronto,0,American Restaurant,Big Box Store,Pizza Place,Cosmetics Shop,Grocery Store
35,Central Toronto,0,American Restaurant,Big Box Store,Pizza Place,Cosmetics Shop,Grocery Store
38,Central Toronto,0,American Restaurant,Big Box Store,Pizza Place,Cosmetics Shop,Grocery Store
39,Central Toronto,0,American Restaurant,Big Box Store,Pizza Place,Cosmetics Shop,Grocery Store
40,Central Toronto,0,American Restaurant,Big Box Store,Pizza Place,Cosmetics Shop,Grocery Store
41,Central Toronto,0,American Restaurant,Big Box Store,Pizza Place,Cosmetics Shop,Grocery Store
44,Central Toronto,0,American Restaurant,Big Box Store,Pizza Place,Cosmetics Shop,Grocery Store


In [121]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels']==1,Etobicoke_merged.columns[[1]+list(range(5,Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,West Toronto,1,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega
13,West Toronto,1,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega
17,West Toronto,1,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega
18,West Toronto,1,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega
23,West Toronto,1,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega
24,West Toronto,1,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega
25,West Toronto,1,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega
36,West Toronto,1,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega
37,West Toronto,1,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega
42,West Toronto,1,Toy / Game Store,Yoga Studio,Fish Market,Curling Ice,Deli / Bodega


In [221]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels']==2,Etobicoke_merged.columns[[1]+list(range(5,Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
9,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
21,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
22,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
23,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
33,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
34,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
35,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
39,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach
43,Scarborough,2,Clothing Store,Café,Hotel,Fast Food Restaurant,Beach


In [122]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels']==3,Etobicoke_merged.columns[[1]+list(range(5,Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,East Toronto,3,Coffee Shop,Bar,American Restaurant,Café,Pizza Place
19,East Toronto,3,Coffee Shop,Bar,American Restaurant,Café,Pizza Place
20,East Toronto,3,Coffee Shop,Bar,American Restaurant,Café,Pizza Place
26,East Toronto,3,Coffee Shop,Bar,American Restaurant,Café,Pizza Place
27,East Toronto,3,Coffee Shop,Bar,American Restaurant,Café,Pizza Place
30,East Toronto,3,Coffee Shop,Bar,American Restaurant,Café,Pizza Place
73,East Toronto,3,Coffee Shop,Bar,American Restaurant,Café,Pizza Place


In [123]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels']==5,Etobicoke_merged.columns[[1]+list(range(5,Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


### hence we conclude that we recomment uses who want to start a  business should open a Coffee shop in East Toronto as it most common venue there 
### likewise we recomment uses who want to start a  business should open a Clothing store in Scarborough
### and Game store in West Toronto and American Restaurant in Central Toronto